# COVID-19 Mortality Prediction with Reccurent Neural Network Deep Learning Model

### Import Libs

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pandas_profiling
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

import z_utils.project as zu
zu.get_py_path()

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

### Call API

In [142]:
import requests

def fetch_data(feature='stringency', start_date='2020-02-14', end_date='2021-02-14'):
    """
    Get lyrics from Seeds Lyrics API. Returns empty string if song not found
    start_date and end_date like 'YYYY-MM-DD'
    """
    url = f'https://covidtrackerapi.bsg.ox.ac.uk/api/v2/{feature}/date-range/{start_date}/{end_date}'
    response = requests.get(url)
    if response.status_code != 200:
        return ''
    data = response.json()
    return data

In [143]:
data_api = fetch_data()

In [144]:
[(k, [c for c in v if c == 'VNM'])  for k, v in data_api.items()  if k == 'countries' ]

[('countries', ['VNM'])]

In [145]:
[([([([(vee)  for kaaa, veee  in vee.items() if kaaa in ['date_value', 'confirmed']  ])  for kaa, vee  in ve.items() if kaa =='USA'   ])  for ka, ve  in v.items() ])  for k, v in data_api.items() if k=='data'   ]

[[[[{'date_value': '2020-02-14',
     'country_code': 'USA',
     'confirmed': 14,
     'deaths': 0,
     'stringency_actual': 5.56,
     'stringency': 5.56,
     'stringency_legacy': 7.14,
     'stringency_legacy_disp': 7.14},
    {'date_value': '2020-02-14',
     'country_code': 'USA',
     'confirmed': 14,
     'deaths': 0,
     'stringency_actual': 5.56,
     'stringency': 5.56,
     'stringency_legacy': 7.14,
     'stringency_legacy_disp': 7.14}]],
  [[{'date_value': '2020-02-15',
     'country_code': 'USA',
     'confirmed': 14,
     'deaths': 0,
     'stringency_actual': 5.56,
     'stringency': 5.56,
     'stringency_legacy': 7.14,
     'stringency_legacy_disp': 7.14},
    {'date_value': '2020-02-15',
     'country_code': 'USA',
     'confirmed': 14,
     'deaths': 0,
     'stringency_actual': 5.56,
     'stringency': 5.56,
     'stringency_legacy': 7.14,
     'stringency_legacy_disp': 7.14}]],
  [[{'date_value': '2020-02-16',
     'country_code': 'USA',
     'confirmed': 14,
 

In [146]:
[(k, [(ka, [(kaa, vee)  for kaa, vee  in ve.items() if kaa =='USA'   ])  for ka, ve  in v.items() ])  for k, v in data_api.items() if k=='data'   ]

[('data',
  [('2020-02-14',
    [('USA',
      {'date_value': '2020-02-14',
       'country_code': 'USA',
       'confirmed': 14,
       'deaths': 0,
       'stringency_actual': 5.56,
       'stringency': 5.56,
       'stringency_legacy': 7.14,
       'stringency_legacy_disp': 7.14})]),
   ('2020-02-15',
    [('USA',
      {'date_value': '2020-02-15',
       'country_code': 'USA',
       'confirmed': 14,
       'deaths': 0,
       'stringency_actual': 5.56,
       'stringency': 5.56,
       'stringency_legacy': 7.14,
       'stringency_legacy_disp': 7.14})]),
   ('2020-02-16',
    [('USA',
      {'date_value': '2020-02-16',
       'country_code': 'USA',
       'confirmed': 14,
       'deaths': 0,
       'stringency_actual': 5.56,
       'stringency': 5.56,
       'stringency_legacy': 7.14,
       'stringency_legacy_disp': 7.14})]),
   ('2020-02-17',
    [('USA',
      {'date_value': '2020-02-17',
       'country_code': 'USA',
       'confirmed': 14,
       'deaths': 0,
       'stringen

### Read CSV

In [149]:
!curl -L 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/c1m_school_closing.csv' > ../raw_data/c1m_school_closing.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1020k  100 1020k    0     0  5510k      0 --:--:-- --:--:-- --:--:-- 5730k


In [150]:
data_csv = pd.read_csv('../raw_data/c1m_school_closing.csv') #,index_col=)

data_csv.head()

,Unnamed: 0,country_code,country_name,region_code,region_name,jurisdiction,01Jan2020,02Jan2020,03Jan2020,04Jan2020,...,27Aug2022,28Aug2022,29Aug2022,30Aug2022,31Aug2022,01Sep2022,02Sep2022,03Sep2022,04Sep2022,05Sep2022
0,1,ABW,Aruba,NaN,NaN,NAT_TOTAL,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,2,AFG,Afghanistan,NaN,NaN,NAT_TOTAL,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,AGO,Angola,NaN,NaN,NAT_TOTAL,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,4,ALB,Albania,NaN,NaN,NAT_TOTAL,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,AND,Andorra,NaN,NaN,NAT_TOTAL,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
data_csv.shape


(263, 985)

In [152]:
data_csv.plot();

### Explore Data & Analyze

In [153]:
data_csv.isna()

,Unnamed: 0,country_code,country_name,region_code,region_name,jurisdiction,01Jan2020,02Jan2020,03Jan2020,04Jan2020,...,27Aug2022,28Aug2022,29Aug2022,30Aug2022,31Aug2022,01Sep2022,02Sep2022,03Sep2022,04Sep2022,05Sep2022
0,False,False,False,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,False,False,False,True,True,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
2,False,False,False,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
3,False,False,False,True,True,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
4,False,False,False,True,True,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,False,False,False,True,True,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
259,False,False,False,True,True,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
260,False,False,False,True,True,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
261,False,False,False,True,True,False,False,False,False,False,...,False,False,False,True,True,True,True,True,True,True


In [154]:
vn_data = data_csv.loc[data_csv['country_code'] == 'VNM'].copy()

vn_data.head()

,Unnamed: 0,country_code,country_name,region_code,region_name,jurisdiction,01Jan2020,02Jan2020,03Jan2020,04Jan2020,...,27Aug2022,28Aug2022,29Aug2022,30Aug2022,31Aug2022,01Sep2022,02Sep2022,03Sep2022,04Sep2022,05Sep2022
257,258,VNM,Vietnam,NaN,NaN,NAT_TOTAL,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
lmr_list = ['VNM']
lmr_data = data[data['country_code']=='']

In [156]:
data.shape

(263, 984)

In [157]:
data.columns # .tail(7)


Index(['country_code', 'country_name', 'region_code', 'region_name',
       'jurisdiction', '01Jan2020', '02Jan2020', '03Jan2020', '04Jan2020',
       '05Jan2020',
       ...
       '27Aug2022', '28Aug2022', '29Aug2022', '30Aug2022', '31Aug2022',
       '01Sep2022', '02Sep2022', '03Sep2022', '04Sep2022', '05Sep2022'],
      dtype='object', length=984)

In [158]:
data.shape

(263, 984)

In [139]:
import pandas_profiling

In [159]:
%%time
# vn_data.profile_report()

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 11.2 µs


### Split train

In [160]:
# -1. Train splitting
# Let's keep the last 40% of the values out for testing purposes
train_size = 0.6
index = round(train_size*data_csv.shape[0])

X_train = data_csv.iloc[:index]
X_test = data_csv.iloc[index:]

In [161]:
X_train.shape, X_test.shape

((158, 985), (105, 985))

### Normalization layer

In [164]:
# 0. The Normalization Layer
normalizer = Normalization()  # Instantiate a "normalizer" layer
normalizer.adapt(X_train) # "Fit" it on the train set

2022-09-06 15:57:16.333268: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

### RNN model architecture

In [168]:
# 1. The Architecture
rnn_model = Sequential()
rnn_model.add(normalizer) # Using the Normalization layer to standardize the datapoints during the forward pass
rnn_model.add(SimpleRNN(units=20, activation='tanh')) 
rnn_model.add(layers.Dense(10, activation = 'relu'))
# model.add(layers.Dropout(0.3))
rnn_model.add(layers.Dense(1, activation = 'linear'))

### Compile model with 'rmsprop'

In [169]:
# 2. Compiling with 'rmsprop' rather than 'adam' (recommended)
rnn_model.compile(loss='mse',
              optimizer='rmsprop')  

In [170]:
rnn_model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

### Train model

In [172]:
# 3. Training
es = EarlyStopping(monitor = 'val_accuracy',
                   patience = 5,
                   verbose = 0,
                   restore_best_weights = True)

rnn_model.fit(X_train,
          y_train, validation_data = (X_train, y_train),
          batch_size = 16,
          epochs = 5,
          callbacks = [es],
          verbose=1)

NameError: name 'y_train' is not defined

### Evaluate model

In [173]:
# 4. Evaluating
# REGRESSION compilation
model.compile(loss='mse', 
              optimizer='adam', 
              metrics=['mae'])

# The fit
model.fit(X, y,
         batch_size=16,
         epochs=10, verbose=0)

# The prediction (one per sequence/city)
model.predict(X)

NameError: name 'model' is not defined